### Setup

In [32]:
import os, sys
sys.path.append('..')

import warnings
warnings.filterwarnings('ignore', category=UserWarning, module='cobra')

from cobra import io, Model, Solution
from scripts.helpers.model import rxn_in_model, met_in_model, add_single_gene_reaction_pair
from scripts.opt._fba import flux_balance_analysis
from scripts.opt._fva import run_flux_variability_analysis

### Import Models

In [34]:
# Load models

# Load wildtype from manual directory (adjust path for notebooks directory)
# Use io.read_sbml_model for local files instead of io.load_model
wildtype = io.read_sbml_model('../data/fill/xmls/MNL_iCre1355_auto_GAPFILL.xml')

models = {
    "Wildtype": wildtype
}

# Get altered reactions of wildtype - using correct directory name
altered_dir = '../data/altered/xmls/MNL_iCre1355_auto_GAPFILL/'

for root, dirs, files in os.walk(altered_dir):

    # Exclude the /h directory from search
    if root.endswith('/h'):
        continue

    for file in files:
        if file.endswith('.xml'):
            # Get file name and remove .xml extension
            model_name = file[:-4]
            full_path = os.path.join(root, file)
            print(f"Loading model: {model_name} from {full_path}")
            models[model_name] = io.read_sbml_model(full_path)

# Print out loaded models
print(f"\nLoaded {len(models)} models:")
for name in models.keys():
    print(f"  - {name}: {models[name].name}")

No objective coefficients in model. Unclear what should be optimized


Loading model: SQE+MVA from ../data/altered/xmls/MNL_iCre1355_auto_GAPFILL/SQE+MVA.xml


No objective coefficients in model. Unclear what should be optimized


Loading model: SQE from ../data/altered/xmls/MNL_iCre1355_auto_GAPFILL/SQE.xml


No objective coefficients in model. Unclear what should be optimized


...

Loading model: SQS+SQE from ../data/altered/xmls/MNL_iCre1355_auto_GAPFILL/SQS+SQE.xml


No objective coefficients in model. Unclear what should be optimized


Loading model: SQS from ../data/altered/xmls/MNL_iCre1355_auto_GAPFILL/SQS.xml


No objective coefficients in model. Unclear what should be optimized



Loaded 7 models:
  - Wildtype: None
  - SQE+MVA: MNL_iCre1355_auto_GAPFILL_SQE+MVA
  - SQE: MNL_iCre1355_auto_GAPFILL_SQE
  - SQS+MVA: MNL_iCre1355_auto_GAPFILL_SQS+MVA
  - SQS+SQE+MVA: MNL_iCre1355_auto_GAPFILL_SQS+SQE+MVA
  - SQS+SQE: MNL_iCre1355_auto_GAPFILL_SQS+SQE
  - SQS: MNL_iCre1355_auto_GAPFILL_SQS


### IDs and Vmax

In [35]:
from enum import Enum

class MetID(Enum):
    # Pyruvate Compounds
    IPP = "ipdp_h"
    DMAPP = "dmpp_h"
    # Sterol Compounds
    ERG = "ergosterol_c"
    EPI = "episterol_c"
    METLOP = "methylop_c"
    FEC = "mfecostrl_c"
    EPOXY = "Ssq23epx_c"
    SQL = "sql_c"
    # MVA Compounds
    HMCOA = "hydmetcoa_c"
    MVA = "mva_c"
    PMVA = "phmva_c"
    DPMVA = "diphmva_c"
    ORTHOP = "orthop_c"
    

class RxnID(Enum):
    # Biomass
    BIOMASS = "Biomass_Chlamy_auto"
    # Exchange Assumptions
    EXCHERG = "ERGOSTEROLEXCH" # This will be added to models first
    EXCHORTHO = "ORTHOPHOSPHATEEXCH"
    # Ergosterol Synthesis
    SS = 'SS'
    SS2 = 'ALT_SQS2'
    SQE = 'SMO'
    SQE2 = 'ALT_SQE2'
    CAS = 'CAS'
    METLOP = 'FILL_HYD1'
    EPI = 'ERG28'
    ERG = 'ERG'
    # MVA Precursors
    ACACT = "ACACT"
    HBCO = "HBCO"
    # MVA Pathway
    MVAS = 'ALT_MVAS'
    MVAE = 'ALT_MVAE'
    MVAD = 'ALT_MVAD'
    PMK = 'ALT_PMK'
    MVK = 'ALT_MVK'
    IDLI = 'ALT_IDLI'
    # Pyruvate Metabolism
    # Acetyl-CoA (Cytosol)
    ACS = "ACS"
    ATPCS = "ATPCS"
    MCDC = "MCDC"
    ACALD = "ACALD"
    # Acetyl-CoA (Chloroplast)
    PYRDHY = "PDHe2hr"
    ACALDH = "ACALDh"
    PFL = "PFLh"
    PYRS = "PYRShi"
    # Acetyl-CoA transport (Cytosol <--> Chloroplast)
    ACCOA = "ACCOAth"

### Empirical Data

In [13]:
Vmax = {
    RxnID.SS: 0.00328,
    RxnID.SQE: 0.0016
}

### Preprocessing Models

In [ ]:
for name in models.keys():
    model = models[name]

    # Add export reactions
    
    if not rxn_in_model(model, RxnID.EXCHERG.value):
        add_single_gene_reaction_pair(
            model=model, 
            gene_id="EXCHERG_GENE",
            reaction_id=RxnID.EXCHERG.value,
            reaction_name="Ergosterol exchange (assumption)", 
            reaction_subsystem="Exchange", 
            metabolites=[(-1, MetID.ERG.value)],
            reversible=True
        )

    if not met_in_model(model, MetID.ORTHOP.value):
        continue

    if not rxn_in_model(model, RxnID.EXCHORTHO.value):
        add_single_gene_reaction_pair(
            model=model, 
            gene_id="EXCHORTHO_GENE",
            reaction_id=RxnID.EXCHORTHO.value,
            reaction_name="Orthophosphate exchange (assumption)", 
            reaction_subsystem="Exchange", 
            metabolites=[(-1, MetID.ORTHOP.value)],
            reversible=True
        )

    # Add Vmax information to model
    # for k, v in Vmax.items():
    #     if not rxn_in_model(model, k): continue
    #     model.reactions.get_by_id(k).bounds = (v, v)

_obj = [RxnID.EXCHERG]
objective = [obj.value for obj in _obj]

SQE+MVA
{'atp_c', 'adp_c', 'orthop_c', 'co2_c', 'ipdp_c', 'diphmva_c'}
SQS+MVA
{'atp_c', 'adp_c', 'orthop_c', 'co2_c', 'ipdp_c', 'diphmva_c'}
SQS+SQE+MVA
{'atp_c', 'adp_c', 'orthop_c', 'co2_c', 'ipdp_c', 'diphmva_c'}


### Flux Balance Analysis

In [38]:
# Run flux-balance analysis on each model and store results
fba = {}

for name, model in models.items():
    print(f"\nRunning FBA for model: {name}")

    # Assume fixed biomass growth
    BIOMASS = model.reactions.get_by_id(RxnID.BIOMASS.value)
    BIOMASS.bounds = (0.005, 0.005)

    # Ignore if objective doesn't exist in model
    if not all([rxn_in_model(model, item) for item in objective]):
        fba[name] = None
        continue

    # Run flux-balance analysis
    fba[name] = flux_balance_analysis(
        model,
        objectives=objective,
        is_pfba=True
    )

    # for _, item in RxnID.__members__.items():
    #     if rxn_in_model(model, item.value):
    #         print(f"\t{item.name}: {fba[name].fluxes[item.value]}")
    


Running FBA for model: Wildtype

Running FBA for model: SQE+MVA

Running FBA for model: SQE

Running FBA for model: SQS+MVA

Running FBA for model: SQS+SQE+MVA

Running FBA for model: SQS+SQE

Running FBA for model: SQS


### Flux Variability Analysis

In [ ]:
fva = {}

target_rxns = [
    RxnID.BIOMASS.value,
    RxnID.EXCHERG.value,
    RxnID.ERG.value,
    RxnID.SS.value,
    RxnID.SS2.value,
    RxnID.SQE.value,
    RxnID.SQE2.value,
    RxnID.CAS.value,
    RxnID.MVAD.value,
    RxnID.MVAE.value,
    RxnID.MVAS.value,
    RxnID.PMK.value,
    RxnID.MVK.value,
    RxnID.IDLI.value
]

for name, model in models.items():
    print(f"\nRunning FVA for model: {name}")
    fva[name] = run_flux_variability_analysis(
        model,
        loopless=True,
        pfba_factor=None,
        objectives=objective,
        reactions=[rxn for rxn in target_rxns if rxn_in_model(model, rxn)]
    )
    print(fva[name])


Running FVA for model: Wildtype
                     minimum       maximum
Biomass_Chlamy_auto     0.00  3.854623e-17
ERGOSTEROLEXCH          0.04  4.000000e-02
FILL_ERG                0.04  4.000000e-02
SS                      0.04  4.000000e-02
SMO                     0.04  4.000000e-02
CAS                     0.04  4.000000e-02

Running FVA for model: SQE+MVA
                          minimum       maximum
Biomass_Chlamy_auto  0.000000e+00  1.243184e-15
ERGOSTEROLEXCH       6.896552e-02  6.896552e-02
FILL_ERG             6.896552e-02  6.896552e-02
SS                   6.896552e-02  6.896552e-02
SMO                 -1.530359e-16  7.717955e-15
ALT_SQE2             6.896552e-02  6.896552e-02
CAS                  6.896552e-02  6.896552e-02
ALT_MVAD             0.000000e+00  0.000000e+00
ALT_MVAE             0.000000e+00  1.206953e-29
ALT_MVAS             0.000000e+00  1.206953e-29
ALT_PMK              0.000000e+00  0.000000e+00
ALT_MVK              0.000000e+00 -1.072847e-29
ALT_IDLI  

### Results

In [39]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

# Define target reactions for heatmap
target_values = [
    RxnID.EXCHERG.value,
    RxnID.BIOMASS.value,
    RxnID.ERG.value,
    RxnID.SS.value,
    RxnID.SS2.value,
    RxnID.SQE.value,
    RxnID.SQE2.value,
    # RxnID.ACACT.value,
    # RxnID.HBCO.value,
    # RxnID.MVAS.value,
    # RxnID.MVAE.value,
    # RxnID.MVAD.value,
    # RxnID.PMK.value,
    # RxnID.MVK.value,
    # RxnID.IDLI.value,
]

target_names = [
    RxnID.EXCHERG.name,
    RxnID.BIOMASS.name,
    RxnID.ERG.name,
    RxnID.SS.name,
    RxnID.SS2.name,
    RxnID.SQE.name,
    RxnID.SQE2.name,
    # RxnID.ACACT.name,
    # RxnID.HBCO.name,
    # RxnID.MVAS.name,
    # RxnID.MVAE.name,
    # RxnID.MVAD.name,
    # RxnID.PMK.name,
    # RxnID.MVK.name,
    # RxnID.IDLI.name,
]

# Create data for heatmap
heatmap_data = []
model_names = []
flux_matrix = []

for model_name, fba_result in fba.items():
    model_names.append(model_name)
    flux_row = []

    for rxn_id in target_values:
        # Get flux value, default to 0 if reaction not found
        flux_value = 0
        try: flux_value = fba_result.fluxes[rxn_id]
        except: pass
        flux_row.append(flux_value)
    
    flux_matrix.append(flux_row)

# Create DataFrame for easier handling
df_heatmap = pd.DataFrame(
    flux_matrix, 
    index=model_names, 
    columns=target_names
)

print("FBA Flux Heatmap Data:")
print(df_heatmap)

# Create heatmap using Plotly
fig = go.Figure(data=go.Heatmap(
    z=df_heatmap.values,
    x=target_names,
    y=model_names,
    colorscale='RdYlBu_r',  # Red-Yellow-Blue reversed (red=high, blue=low)
    text=df_heatmap.values.round(4),  # Show values on cells
    texttemplate="%{text}",
    textfont={"size": 10},
    colorbar=dict(title="Flux (mmol/gDW/h)")
))

fig.update_layout(
    title=f"FBA with Objective: {', '.join(list(map(lambda x: x.name, _obj)))}",
    xaxis_title="Reactions",
    yaxis_title="Models",
    width=800,
    height=600,
    font=dict(size=12)
)

fig.show()

FBA Flux Heatmap Data:
              EXCHERG  BIOMASS       ERG        SS  SS2       SQE      SQE2
Wildtype     0.036166    0.005  0.036166  0.036166  0.0  0.036166  0.000000
SQE+MVA      0.062357    0.005  0.062357  0.062357  0.0 -0.000002  0.062359
SQE          0.062357    0.005  0.062357  0.062357  0.0 -0.000002  0.062359
SQS+MVA      0.036166    0.005  0.036166  0.036166  0.0  0.036166  0.000000
SQS+SQE+MVA  0.062357    0.005  0.062357  0.062357  0.0 -0.000002  0.062359
SQS+SQE      0.062357    0.005  0.062357  0.062357  0.0 -0.000002  0.062359
SQS          0.036166    0.005  0.036166  0.036166  0.0  0.036166  0.000000


In [ ]:
# Export results
import os

# Create results directory if it doesn't exist
results_dir = os.path.join('..', 'results', 'bench', 'fba_flux_heatmap')
os.makedirs(results_dir, exist_ok=True)

# Save the heatmap as PNG
output_path = os.path.join(results_dir, f'obj_{"_".join(objective)}.png')
fig.write_image(output_path, width=800, height=600, scale=2)
print(f"Heatmap saved to: {output_path}")

# Also save the DataFrame as CSV for reference
csv_path = os.path.join(results_dir, 'fba_flux_data.csv')
df_heatmap.to_csv(csv_path)
print(f"Flux data saved to: {csv_path}")

# Save the heatmap as HTML for interactive viewing
html_path = os.path.join(results_dir, 'fba_flux_heatmap.html')
fig.write_html(html_path)
print(f"Interactive heatmap saved to: {html_path}")

# Save the heatmap as .png for convenience
png_path = os.path.join(results_dir, 'fba_flux_heatmap.png')
fig.write_image(
    png_path,
    width=1200,
    height=800,
    scale=2,
    format="png"
)
print(f"Image (PNG) heatmap saved to: {png_path}")

Heatmap saved to: ../results/bench/heatmap\obj_ERGOSTEROLEXCH_Biomass_Chlamy_auto.png
Flux data saved to: ../results/bench/heatmap\fba_flux_data.csv
Interactive heatmap saved to: ../results/bench/heatmap\fba_flux_heatmap.html
Image (PNG) heatmap saved to: ../results/bench/heatmap\fba_flux_heatmap.png


### Save Temp Models

In [7]:
temp_path = os.path.join('.', 'temp')
os.makedirs(temp_path, exist_ok=True)
for k, v in models.items():
    v.objective = RxnID.EXCHERG.value
    io.write_sbml_model(v, os.path.join(temp_path, f"{k}.xml"))